Brute force algorithm for motif finding

In [1]:
def HammingDistance(p, q):
    mismatch = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            mismatch += 1
    return mismatch

In [3]:
def Neighbors(Pattern, d):
    if d == 0:
        return {Pattern}
    if len(Pattern) == 1:
        return {'A', 'C', 'G', 'T'}

    neighborhood = set()
    suffix_neighbors = Neighbors(Pattern[1:], d)
    for neighbor in suffix_neighbors:
        if HammingDistance(Pattern[1:], neighbor) < d:
            for nucleotide in {'A', 'C', 'G', 'T'}:
                neighborhood.add(nucleotide + neighbor)
        else:
            neighborhood.add(Pattern[0] + neighbor)
    return neighborhood

In [4]:
def MotifEnumeration(Dna, k, d):
    Patterns = set()
    strings = Dna.split()
    for i in strings:
        for j in range(len(i) - k + 1):
            k_mer = i[j:j+k]
            neighbourhood = Neighbors(k_mer, d)
            for neighbour in neighbourhood:
                found_in_all = True
                for m in strings:
                    if not any(HammingDistance(neighbour, m[n:n+k]) <= d for n in range(len(m) - k + 1)):
                        found_in_all = False
                        break
                if found_in_all:
                    Patterns.add(neighbour)
    return Patterns

In [5]:
import time

with open('motif_enumeration.txt', 'r') as f:
    data = f.readlines()
    k, d = data[0].split(' ')
    k = int(k.strip())
    d = int(d.strip())
    Dna = data[1].strip()
    start_time = time.time()
    result = MotifEnumeration(Dna, k, d)
    end_time = time.time()
    print(f"Time taken: {round((end_time - start_time), 2)} seconds")
    with open('motif_enumeration_output.txt', 'w') as out:
        out.write('\n'.join(result))

Time taken: 0.02 seconds


**Compute the entropy of the NF-kB matrix**

Entropy = -p . log 2 (p)

Entropy is an indication of conservation calculated from the entropy of each column of the matrix. The more conserved the column, the smaller its entropy.

Entropy is a better metric to score motifs

In [15]:
import math

def entropy(strings):
    dna = strings.split()
    num_dna = len(dna)
    entropy_cal = 0
    a, t, g, c = 0, 0, 0, 0
    for i in range(0,len(dna[0])):
        for j in dna:
            if j[i] == 'A':
                a += 1
            elif j[i] == 'T':
                t += 1
            elif j[i] == 'G':
                g += 1
            else:
                c += 1
                
        a /= num_dna
        t /= num_dna
        g /= num_dna
        c /= num_dna
        if a > 0:
            entropy_cal += -(a * math.log2(a))
        if t > 0:
            entropy_cal += -(t * math.log2(t))
        if g > 0:
            entropy_cal += -(g * math.log2(g))
        if c > 0:
            entropy_cal += -(c * math.log2(c))
        a, t, g, c = 0, 0, 0, 0
    
    return entropy_cal
    
entropy("TCGGGGGTTTTT CCGGTGACTTAC ACGGGGATTTTC TTGGGGACTTTT AAGGGGACTTCC TTGGGGACTTCC TCGGGGATTCAT TCGGGGATTCCT TAGGGGAACTAC TCGGGTATAACC")

9.916290005356972

In [2]:
def DistanceBetweenPatternAndStrings(Pattern, Dna):
    k = len(Pattern)
    distance = sum(min(HammingDistance(Pattern, text[i:i+k]) for i in range(len(text) - k + 1)) for text in Dna.split())
    return distance

DistanceBetweenPatternAndStrings("ATGATTA", "CCACGTCGCGAACCATACCCGCCATCGTTCGTGCGGTTCCTAGTACAGGGACACTGACGAAACTAATATAGGAGCGAGTAGTCTTGATATTCCATTGACCAAT AGATTTGGTTGTTTCGTGGTGTCGAATTGGTCGCAGATGATTCGCCTGGTAGGCGATTTCACCCTTGCCCCGTTTAGGGAAATACTAGCCACTACACTGATCT TAGACCAATATTGTAAATTGCGCTGCGGCAGTGCGATTATCGGCGATGCAAGCAAGGGGGCAACTCTCATGAGCATATTTGCTTCGCAAGCCTCCAGCAGTAT GGGGCCCAGCTGATGGCTGTGGGTGTGTGGCATTAAACAGTATGTACTTAATGACAGGTGCGTAAACGCATCATAGGTTTACGTTACGACGTTCTCTGCGAGA GATGTTAGCATATTCCCACCAGCCAGACCACAGAATACGGAGACCACGGTGCTCCCTCCATGAACCCGAGTGCTAGGTGTACGACAACAAAACACTTAAGGCA CCTAGAGCGTTAGAAGAGCAAGCGAAATTTTAAAAAATACGCTTTGTCAACAAACTGCGCAACAGCATATGGCTGTGCGGGAGTAGGAAGTGATATGCGTTTC CAATCATATCCAGATAGTGGTATCAGCAGTGGTGCAATGGGCAGAGTCAGGCATAGATCGGCGCGTAAATACTTATCGTGCCTAGGTTGACGATCCATAGCTA GACTCCACAATGAACAGCGAGCTTGGGTCTATTTGGCGGTCTAACCTTAATGTTTTCTGAACGCTTTTTTTACTCTTCACAGAGCATACAAACCACCGATAGA TACCATTCGAGATGAATGTCAGCCAAAATCCCGTCTGTCCAAAGTCCCGTGACTTATCGCGCAGAATTCAAGCTGTTTATGACCGACGATACAGATCATCTGA GATGATTTATTGGCGATCCTAGTGAATGTGTAGTAGGGGCAGCTTTAGGGAAGTCAAATTCCTAGAGTTCGAAGTTCTGTCTACAAAAATCAGATGTCGCTCG TCAGTTACGTACCGCAGGTCAGCTGAGATGTAATAGGCTCCACCTGTTACGGGCACTCTGGATCGGTGTAAGAATTGGGTACACAGGTAACGGCGAGTGTAGA TCGAGAACCCAGACTGACTTGAGCTGACGGCTTCGGTACAGATGAATTACGGTATCACGTCTATGCTGTCGCCATTAGGCTCAAACCGTAAAGAAGACATGTG TCGTGGTATCATCAAAGACTAATTCTCGACAGGACAAACGCGACTGGCGGATAATAGGATACGATTGGCGATGTGCCTACGTGGATTCGGCGGGGTCAGGCAT GTCGGGATCCTTATCTTGGCATTTCATGCCCCAGAATGCATAGCCATTCACCTTAGTAAAAATGCCCTTACGTTGCACTCTTTCCAGGCGCACCGCTGCGCTC GAGCAGTTGTAAGCCAATTACTTGGGAGAGTAGGCTAGTGATAGCCGCACATGCAGAACCGATTTGGCCGGTAACCTATTACCAGACATCTGACTCGTATGAA GACCGCCCCAAGTCCGTGGACTTTAGCTCGGATCCGTACACCTTAGCGTCACTTCACGCCCTTCCGACCGCGAGGGTAGATTACCTGGCGCCTAGATAATCAC TAACATCCCCGCGTGTTGGGACGATGTTCCATTTATGGGAAAGGCACAAGAAATCTGAAAGTAACTAGGGTAATAGATCGTCGTGTCATACAATTTTTAACGT TGCCAGGGTAACTAGAGCACCGTCTAGGCAACGTCAAACGTGACAGGTGATGCGCCGCTTCTACGAAGTGTCTAGCCGACACTTCGGAATAACGACTGCACTG GGATTTAACGTACAGGAAACCGTCTCTGCTGCCAAGGCCCCACAAACTACTCAGAGAGAAAGTGTCGTGGCCAGCAGAGAGGCGTCGGTTGCATGCGAATCGG GGAATGTAACATAATGCCGTACCCCCAGGGATTATTTGTGAAGCATTTCTACTCACGAATCGGCCTTTAGCCCGAAGCAACGTATGGTCATACTTTGCTAAAA TTTAAGTCACTCGTTACCTATGACTTCTGGTCGTATATCTATATTACCTGTCGATGAAGATTGCAAGAGTGGGCAATTCTTACGTTCTCAATGGAGATATATT GTACTAGTCACTGCTGGCATATTCCAGAGGTTTGGGTTTAAACAGTCCAGCGGGGGTCTCCACTTGCCCGTAAAACCGCCCAATCGATGTCATTTTGGGATTG TGGTTCTTTCGGCTCGGAATCCTATCTTTATAAAAATAATGGCAGACCGTTTACGTTGCTAGCTGTTTTCTCCACTCAGAAATGTGGACAGCAGACGGGAGTT TTGGGTTGTGGAGAGCAATGGTTCAATAGTTTCTGGACGTTGCGAATGGTAATATGTATCACGACCGTCACGAATCTTTCACGGCATGATGCGATCGTGTTAA TATTCAAGCCCTATCCCCGGAACCATTTTCCACGTGGTATCAAAGGTCTTGTGCCTCCGGACACCATATGCGGCACATGGTTTTCCAGCTCAGATTGTTCAAG CCACCTGTACCCAGAGTATGCGAAACCTGACGGTCGCCCAGGTTCGGTTTCAAGGAGACCTGGCTGACGGATTTACAAAGGCTGTTTTTCTACTATGGTACGG GGACACATTTGACGGTCTAGTCGCGCCATGCCTCACTAACGCTGTTCGAATGTTCTCAATTGATTCGATTTCACGCGAGCAGATGTCCCTGCGTAACAGTCAG CAAAGTGGCCATAAACCTGATTGAGGCACTGTCCTCCGTAGGGGGGTCTATTGATCTGTCCGACCGTAAAGACAAGAAAGTCCTGTGTGAGACAACCGACAAA TGTTAATACCGAATGCCTACGGCGGCTCGGAGGACGAAATTAAGACCTCTGGTTGCCTCGAGATTGTGTTGTCTACCGCGGCTCAATACAGGCAAATCCCGGT GACTACTCTCATGTGGTAACCTTCATTATAGGTGCTACTGGTTGACGGGTGAGATGACGAATTGATAACGGCAGTGTCTGACACAGTAGTATTGAACAACGTC")

67

In [59]:
def median_string(k, dna):
    distance = float('inf')
    pattern = ""

    for i in range(len(dna[0]) - k + 1):
        current_pattern = dna[0][i:i+k]
        current_distance = sum(min(HammingDistance(current_pattern, seq[i:i+k]) for seq in dna) for i in range(len(dna[0]) - k + 1))

        if current_distance < distance:
            distance = current_distance
            pattern = current_pattern

    return pattern

median_string(6, ["AAGCTTTCCTTTACAGTTTTCCAGCACAGCCGGAGGTGAATG", "GAAAGGACAGTTAACGTGACTGACGCGCTCCGGAATATTAGA", "ATCGGGTAGGTTGAGGTGAAAGTTCTAAACAAAACGTAGACG", "GGACCATCACGGCCTGGATGGCGAACTGACAGAGTTGCTTTT", "TGACCGAGTTCTTAGAGGTTCCGGACAGTGAACCCCAGAGTT", "GAAGTGAGAAAGGACGTTATAGTTCCTAAATTCAGAACCATG", "ACAGTTAAGTGTCCTCGGGGTTTTGTACACTGAGTCGTTGGA", "ACATAAGTAGCGGAGTGAATGACGTAGCAGATAGTTAGGGTT", "TAAATCAATTTAGTTCCGATAGTTGGCTGTTTCGCAAAATCA", "CGCCCGCCATCAAACTCTTCGGGCTTACCGTTTGGGAAAGTT"])

'AGTTTT'

In [5]:
import pandas as pd

def Profile_MostProbable_kmer(string, k, profile):
    kmer_score = {}
    for i in range(len(string) - k + 1):
        kmer = string[i:i+k]
        score = 1
        for j in range(len(kmer)):
            score *= profile.loc[kmer[j], j]
        kmer_score[kmer] = score
    
    max_score = max(kmer_score.values())
    print(f"Most probable kmer is {list(kmer_score.keys())[list(kmer_score.values()).index(max_score)]} \
          with score {max_score}")
    return list(kmer_score.keys())[list(kmer_score.values()).index(max_score)]
            
data = [[0.277, 0.265, 0.265, 0.229, 0.217, 0.313, 0.229, 0.313, 0.277, 0.181, 0.193, 0.229], [0.265, 0.301, 0.265, 0.253, 0.265, 0.193, 0.157, 0.253, 0.277, 0.229, 0.337, 0.217], [0.217, 0.241, 0.277, 0.325, 0.289, 0.229, 0.313, 0.181, 0.205, 0.289, 0.205, 0.253], [0.241, 0.193, 0.193, 0.193, 0.229, 0.265, 0.301, 0.253, 0.241, 0.301, 0.265, 0.301]]
df = pd.DataFrame(data, index = ['A', 'C', 'G', 'T'])
Profile_MostProbable_kmer('TTGGACATAACGGTAGTGCATAACCGTAATACTTTATAACCTTCAGATCCACCACAACAGCAACCGCACTTATGCTCCCAGTCACCAATGATAGCAAGACGGCGCTCGCCTCTATGAAATGTCTCGCGAGTGCCTATTAGAGGCTCTTACAGTACACAGGTCAGACGAAAGGTCCAAGTATTCCGTGTCCCTACCAAATGACCCTGGTTTAGGCTGAAACTCAGGCCTAATAGCTCACGCTGTAACGTTAGCCTTGCCAAGGATCATAAGGAGTGCCCGCATGCCGGGTCAGCCTATACAATTGAATCTAACGGCATACCTCTAGATTTCAAAGAAGGGGACGGGTCCCGTCAAAGTAAAAGAATACGAGGGATCGTCACTATATTGCTTTAAGAGGTTCTTGGTAGGTACGAATTCTTCGACTATAAAGAGAGCCTCACCGCTAACAACCATCGAAAGAGAAACATAAAATTGATCGGCCGACAGAATGACATCCAAACAGTGCACGGATGCCAAGGGTGCTTGGGACCTCGAGCTGAAAGAGCCTGAACGTATTATCTGACAGCTGAGTCGATTCATATGAGAGACCTCAACTAAGTGTCGCTTAGATAGCAACCGAATAGGCCCGTGCAAGTGGGCTTAACTATCTGAGGAGATCTGTAGGACTTAGGAAAGCGGGCGTGGGTGCTGCTGTCTAGCCCGATGAATCGTAACGAGATCTGAGTTATGCAAGAGAATGTTGTTCAGCTCCTTTGGGGTCTCACTTCTGAATCCGGTAATTGTTTCTGACCCCCTGATATAACAAGATCTTATACATTTTTAACAGCTACCTAATCGCGAATCAGACCCCTGCTCCGGTTCGGTTTGCAACTGTGGCTTCCCAAGGGAGACGTCCGGAGATTTTGATCTGCGGAATAAGCCCAGATCCGAATAACATAGCGCACGCCTCCCTCTTACTAACCGAGTTCCATATGCACAGACGTTTTACGTGCGTTCTCAC', 12, df)

Most probable kmer is TCCGGAGATTTT           with score 3.2036092294833274e-07


'TCCGGAGATTTT'

**Greedy Motif Search**

In [13]:
def profile_most_probable_kmer(text, k, profile):
    nucleotides = {'A', 'C', 'G', 'T'}
    max_probability = -1.0
    most_probable_kmer = ""
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        probability = 1.0
        for j, nucleotide in enumerate(kmer):
            probability *= profile[nucleotide][j]
        if probability > max_probability:
            max_probability = probability
            most_probable_kmer = kmer
    return most_probable_kmer

In [14]:
def form_profile(motifs):
    k = len(motifs[0])
    profile = {nucleotide: [0] * k for nucleotide in 'ACGT'}
    t = len(motifs)
    for i in range(k):
        column = [motif[i] for motif in motifs]
        for nucleotide in 'ACGT':
            count = column.count(nucleotide)
            profile[nucleotide][i] = count / t
    return profile

In [15]:
def score_motifs(motifs):
    k = len(motifs[0])
    t = len(motifs)
    score = 0
    for i in range(k):
        column = [motif[i] for motif in motifs]
        max_count = max(column.count(nucleotide) for nucleotide in 'ACGT')
        score += t - max_count
    return score

In [65]:
def greedy_motif_search(dna, k, t):
    best_motifs = [seq[:k] for seq in dna]
    for i in range(len(dna[0]) - k + 1):
        motif1 = dna[0][i:i+k]
        motifs = [motif1]
        for j in range(1, t):
            profile = form_profile(motifs)
            most_probable_kmer = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(most_probable_kmer)
        if score_motifs(motifs) < score_motifs(best_motifs):
            best_motifs = motifs
    return best_motifs

Dna = "GATTCAGTCTACGGATAGCAATTGCTCGATGGATTTTACGCCGCACAATTGACTATGGATGGCTCTGTTGCATTTTGATATTCTTGAGCCATCACGAACTCTTTGCGACTGGCAATTCAATCAGTCTGCCCGTGTACAAATTAGTCAGCGCGGTGA GTTTACAGACGATGAATTAATCCCCAGCTCTTACTCCCGAGTAGAGAATAAGTGCCTCCACTATGGTATACAAGTTGTTAGGTGCCTTCAATCTACTAGACCTATGATGGCTCTTCGTCAAATAAACTAACAAGGAAGGTCCTTGCAGTTACTAGA CATCTAACGTCCCATTCACTCTACTGATCGTTCAAAACTGACTGTCGAGCCACCTACTGGTGCTCTCGGCTGTTGCTAACATTACCGTGCACATGCACCTTAGCGGTCGTCGATCAGAATGATTTGTCCTCCCGTGTCAGGTGTGGTTAAGTCTGA TGGCCCCAACCTAGCAATGGAGCTAGATGTTCACCTCAATGAACCTACCCCCGCCAGCTGCTTCAAGCATAGCCTCACTGCCTACAGCCAGCTGTAAGGCTTGGCAGCTGTTCAATCAACTTACGGGTCAAAGGTAGTGCAATGTCGCGATCCTAG ATAAGTCCGTATCGTTCCAATTTTGTTACACAATTAGAAATTGGGCTGTCAAGCTCACTTTAAGAACCGACCACAAGGGCTGGCGCATATTGGTTACGAACCTCGGTGTTTAGCGCCTCCCATTCATTCTACGCAGTGGGACTAGTTCGTAGTACG CCTGGAGTACACGTTACGTGTCAAACGTATTTAGGGAAGACGCTCTGCCGTGATCTCGAAAGTTCACTCAACCAAGTTACGATAGCGTAGCTGATGCACGCACTTTCGTACTATCCAGGGCGGCGGCCTGCGCTAGTCACCTAGCTCATTGCCCAA CCCAGTGCCGGGTCGATACCGCCCCTGTTAAAGACTTCTCATAATTAAGCCTCCTTAGCAAGCCGAGATACGGACACCGGGCCCTTGCGGGCCTGCAAGTGGCACGTACCCACAGCGCAGCTAGACCTCTTCATTGTAAGAATATCTTCAATCGAC TCATCCGATCGTCACAACTGTTCATCTCCGGGCCATTTGGTTATAGTTTATTCAATCCACACCACCTCACCAAGGGAATGAAAATACCGAGACGGATGCGGAATGATGCTATGGGCAATTTCTAGCATAGTGATTGGGATGCTATATGGGAGGCGA TCACACGGTGCCTGCGTAGGGTTAGGGCAGCGGGTCTGCCCTACACAAAGGTAGACAACATGGATCTCAACCGGTCCCCACTTAGACTAGAACTCTCCGCCCGAGCCAGTTTCAATCAACGAGATTTTTCCGGACCTATTCTCGCCTCTAGTGTTA AGCTTCCCTGACCCTCGGTGGATAGTACCGTTAATACTCGGCGTCTTGTGCGACGGTATTGGTTCAGTCTACCGTTCTCGGGGCTGTACCCACCTAATTCGTGCCTGAGTCTCCCAATCAGCCTGGTTCTGGCCGACAACAGCAACTGTGCGCGCG ACACGGTTAGCTGATCAGTCTCAAAACACAGCACGTCACAAGGGCGAACAGCAGAGCGAGGGAATCGAACGCACATTGGTTTGTCTGAACCCCGCCCCTTCATTCGACTGACGAGTACGACCGAACTGTAACCTTAGGTGCTTAGGCCCATAAACA GCTTTAAATTCAGGCAGGAGGCATGTGACGATTTTAAAGCGACTATTTGCGCACACGTGCTTTAAGCAGTATTTTTCAATCCACGGTTGATGCGTCCAAGTCAGTTTCAAGACCTAGAAAATATGGATGATATTCCACCACAGCCTCCGCGTGATA CGAACCCTTTCGCCAATATATGACGACCACCGAAATAGTTCAGTCGACCCATCTACGCTCATGTGCGGAGACCAGCGCTGGTGACCTCTGAATTATGGAGACTCTAATCGCGACGACATCCCCAAGAATATGTCATGAAGGGCCAGGTCAATGCAA TCACTGCGGGCCACTTCAGTCGACAGCGTGTATATTGCCTAGATTCCTCTTGAAGCACTTGAGATAAATAACGGATCTTATCACCGGATATTTTGCTATCCACCTCATACCTTTAATTAAACCTTTTATCGGAACGCAGGGTGTACTCGGACTAGG TTCTGACTCTACGAGGGGGACCGCTTGGCTTTCCTTTGAGGCGGTTACATTGTCATTTTGTATTCAGTCTACCCGGGGGCCGACTAGTGGAGGCGGTGGCACATTCGGGCCATTGTCTACACAGTTGCTCGGGAAGCGGTGTTCAGATCCTGTTTA CTGGTTACACTCCGTTGTCGGTCCTAGGCTGTGATGTCTAGCCCCTGAATCAAAGCATGCCAGGGTTTCCAAGAGTCGGTACGTGTGCACCGTTGCCTTAAGTCTTAGTTCACGCCCTGGCTGAGAGTGTGCGCTATTGCAACTGGTTCATTCAAC CTAAGATTCCTTGGCCGTACATAAAGGCGCTGATCGACGCTGCCTTGGTGCGTACGGTGAGAAGCGTGCTTAATTTCAGATATCCCCTATCTGTCTGGATGTTATTCTCATTCATTCAACGTCGGTGGGCATCTCGTATGTTCAGATGGGATAGAC TCCACTCGTTCCATCAGCCGGTGGCCCTCGAGAGCTTTAGTGCTGACGAGAGGTGTCACAGGTTCAATCCACTCCTGTTTGGATTGTACAAGGGGCCAGGCTGGTTAGAAATGTTAACTCTCGCTACATCTGCGCGGGACGGCGCTAGTCAGTCGC CGGGAGGATCTTAGTCCCAACATTGATGCGGCTATAAGGAAGATAGGGCTGCCGTTTGGCACGTCTCGTTATTCAAAGCAAAATTTTGAGGCCATCCCAATAACCGTACCACAAGACAGAGTTTACTTCTCGGAGGGAAGATGCGCTTCATTCGAC TACGCGCCGAGCACCTGTGGAGTGGGTGTGGGCAAGACCCTAGTTACATGCGGCACGGGATTCTGGAAGTAAACATAGTATGCTCTGACACAACTTGATGACGATAACTATCCACAGAGCAAAATGATTCATCATCTTTTCGGGATTTCAGTCTAC CTCCACTAAGTACGCATATTCCAAACCTCAGACATCATACATGTATAAGTCTTATATGTTAGATAGGTTAAACCTTCAGAGTTTTTAGGCTAACTCTATAGGTTGAGCGGCATTGGACAGACACGTGGAGGCAATTCAGTCTACTCTAGGACCCGA TTGGGCATCCCGCTTCAGAGGGCCTACCCTCTTGGCACTGGGACGTTAAGTTCACTCTACTTAAATGCCCATTGGTGGGATCCTCCTTCTAGAATAACCGAACAAGCATTGTAAGTTATTCTTAGGAAAGGCACGTATGTCCAGTGAAGGGTCATC GAACCGGCCGCGCCTCTGGCAGAGTGCCGCTATCAGGCGCACCCCGCCATCCACATGTGGGATGGAGCTAATGAATGGCAACTAAGTTCACTCAACGTTGATTTCTCGAATGAATCCTGACCTGCTTGCCAGGGCGTGATTTATCATTTGGGTGAT TGGCTACATTAGTAACAGATCGAGGACTAACTGATGATTTCAGTCAACCTTGCCGGAAAGTGCGGGTGACATCCAAGAGTAGCAAGTGTGGACTCACGAGGCTTGAAGGATACTGCCTGGCGATCTTCATATATTGGCAGTCAACCTCAGTTCGTA AATCGAGTGGGGCTATTCCACAAATAAAACAACCATTAAATTGGCCCCGAAATCTAGACACAACCAATATAGAGTTCAGTCTACTACAGTTGGTCTGTTTGGCGTACCCGATCTCCGCCTCATTTAATCTTCGTTTCTAGAGATTTTCTCAATCGC".split()
k = 12
t = 25
best_motifs = greedy_motif_search(Dna, k, t)
for motif in best_motifs:
    print(motif)

TTCAGTCTACGG
GTTTACAGACGA
CATCTAACGTCC
TGGCCCCAACCT
TTCATTCTACGC
TTCACTCAACCA
TACCCACAGCGC
TTTATTCAATCC
TTCAATCAACGA
TTCAGTCTACCG
GTGCTTAGGCCC
TTCAATCCACGG
TTCAGTCGACCC
TGCTATCCACCT
TTCAGTCTACCC
TGCCTTAAGTCT
TTCATTCAACGT
TTCCATCAGCCG
CGGGAGGATCTT
CTGACACAACTT
TTCAGTCTACTC
TTCACTCTACTT
TTCACTCAACGT
TTCAGTCAACCT
TTCAGTCTACTA


**Improved Greedy Motif Search using Laplace Rule of Succession (Using Pseudocounts)**

In [16]:
def form_profile_pseudocounts(motifs):
    k = len(motifs[0])
    profile = {nucleotide: [1] * k for nucleotide in 'ACGT'}
    t = len(motifs)
    for i in range(k):
        column = [motif[i] for motif in motifs]
        for nucleotide in 'ACGT':
            count = column.count(nucleotide) + 1
            profile[nucleotide][i] = count / t
    return profile

In [17]:
def improved_greedy_motif_search(dna, k, t):
    best_motifs = [seq[:k] for seq in dna]
    for i in range(len(dna[0]) - k + 1):
        motif1 = dna[0][i:i+k]
        motifs = [motif1]
        for j in range(1, t):
            profile = form_profile_pseudocounts(motifs)
            most_probable_kmer = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(most_probable_kmer)
        if score_motifs(motifs) < score_motifs(best_motifs):
            best_motifs = motifs
    return best_motifs

Dna = "TCTCTGTTACTGTCGCTCTCAATATCGGAGGCGTCATCCTATAAGACGGTTTGTAAGTGACGGAGAAATTCAAATGGATTTGTCTTGGAGCGGTAAAGATACACTAAGGTATGTACTATGAAACCGTTCATATGTCGATTGAGTTTACAGGGCATC CCTCTGTGAGCGACGGTCCAAGAACTGTTAATACTTTTCGTGCTAGGGTTTTGCGAGGGTATGTCTAGTTCATCTCGCAGCCAATGAGCTGCGTTAATTCGTATACTTGTCTATCCGAATACGGGAATCTAGAGCGCGGCATTTCCCAGCTCTTCG ATCGGCTCCTCGCACCTAGCGCCTCCCCTCGAGGCATGGGTATTCAACATCGAACGCCAGACTCTGTAAATGCCACACCATGTAATGTACTAAGGCCGCCTGGCATACCTAATCGTTTTGGGCCAGCGGCTCTGCAACCAACTGAACAACGTGACT GTCAATTCCGCTCTACCCCCTCGCCCAAAAATCCAGTTGTGGATGGCTACTCTGTCGGTTGCCTCGCTGTGCTAGGGAGACAATACTCTGTCAATGCCGAGCGCGTGGTGGGTCGCATGTTTCAGCGGGTTCGGTGCTTCAGCTAGATCAAAGACT GATAAATTGAGACTTGTGATTAGGCCTCTGTTATGGAGGAACAGAGTTGCGCGAGGAACGTCAGAGTCTGCATGCCTCGGCCATCCATTTGGCGTTCAGTGTTATCATATCGTCTGATGATCTGCTGGCGAGTCTATCCCAGAAGACCAGACAGTC GTCGAGGTAAGACAGTGTAGGTGTGGTAATGATGAGCAAACTCTACGTTCCATTTCAGTTACTCTGTAAATGAGCTGAAAGTGGATGTATGGCGGGTCTCCTCTTGGGCATAGGAGGCGGTTTTAGATCAGGCTCATTGACGTCCCTGTCCATTGG CCGGATCCGTCCGCTTCGACGTATCAGATGTAACATTTCGGCCAACTGGCTCTGTTAAAGCGACTTTTCATATCAGGCTAACCAGCCGTAAGTACGATAATATCCGGACACGAATTCACAGGGTTTGCACACATTCTTGTATCAACTAGCATGGCA GGCGTCAATTGAGGAGCTTCGCCCGCTCTGTCAGAGCCTTTTCAGATTCCTGGGACCGGTACACGTTAGTATTTAACATGATTCTGCCAACAAAAGGGCTAGGATAGCGTCTCGGTGCCCGGCTGTGGTTCGTGTCCTTAAGTCTTGAGTATGTCA CCTTGACTCAAGAAGCTCCTCGATCGATGGTAGTAGTTCGGTCCCGGGGCTCTGTTAGGGAGTTGATGCTTCCCCCTCGTGGTGAAGCTAAAAAAACACCGTTGTCTACCAATACGCTTGCGCCTCACATGCTAATTCCTAGTCGGCACGGAGACC AAAGCCTGACCCACTCTGTAAATGGCCGGACACGGCGTTGATTACAGTAGAAACGGTTCCCGCCTTTGCCGTAATGCAGCTGGAGCTAGAGTGCCCTGCAAGATGGCTTATCGATATGTCAGTGCTGCACCTGAGCCTCACAGTACCTGCGATTCC ACCGCGGCACGATGTTAACCTATCTATTTGCCCAAACAGTAGCTCCTAAATGCGTACAGTCTACGAACATCTCCTCTGTGAGAGGAACTCGAAAGCAATACCTCGGTAGAATTTCAGGTGCTCACTTCCAAGTACCGCTCCAGGGGACCATATACC CCGGTGCCAATGCCCAAAATAGATTGCGTCTCGGAACCGTGTTGGACTAGGTAGATCTGGGTCGATAAGCCGTGACGGGTCCGCTACGAGCTAGAGTCCACCGTCTGTCCTCTGTCAAGGTGAATACAGCGTTGGCATTAGAACCTATGGTTAAAC GCTCTGTGATTGGCCAACGTGTAACGCTCGGTCAGAACAATATCGCAAAATTGTGTGAAATGACTCCATACTACTTATTAGTGCGGGCATAGGGTTGGTAGGTGGCGTGGACTGCCCAAATGACTTCTAACCTCCCCGGTACCTCCCGGATCTGCC ATTCGTCGAGGCACTCTGTGAGCGTATGAAGCAGAGGGCATGTTTTAGTACGGTGACTATTTGCGGCATAGTCTGTCCTGAAAAACATCTAAGTGAAAGTCCCTGTGTCTTATAACCAGCGAGAACGTACTGTCTATATCTCGCTAGAAGGCCAAC TCTCTGTTAGGGCCTGCCAGGCTAAAATTCTTCTTTTCACTACACCTGGGACGAAGCCGTTTCGAACTATGGCGAGCTCCGAACTAGATAAACTGCTCGTTAGGAATGCGTCCCACCCGGGAATACATAGCGGCCAACCATCGAGAATTAAAAATT CTACGTGCGCTCAACCCTGGTATCCAAGAGCGAAAACTCTAACGGTTTCTATACGCCGTAGCTCTGTGAGTGGCTGCTCGCGTATGATCTGGCCGTTCTCAAGTTGCCATTGGCGAAAATCGGTAGGCTCCTCTCCCCCCGCTGCGCCTTCACATC ACTACCTGCTTAAAGATGAGCGTGTTGTGGTAGCTGTTCACTCTGGGCCCTAAGGGGCGAACCAAGCAGAACACTCTGTAATCGCTTATGGTCTGGTGGAGGAATTAACGACGCAGTAGGATGTTCAGACTGCCATGCGGTTACAATATTCTTATA TTGTGCGCCCGCTTGTCTAACTTCGAGTCAGTTGTCTTGGGAGGCCTTGGAAGGTCTCCGGAGTGGATGCTTGAAGAAAGCGTGACGCCAAGAAATGATGATTAGTACTGCGTCCGACTTAGTAGCAGTGTATCCAGAATGAGGTCTCTGTAAGGG TCGGTTTAGCTGGCTCTGTAAGCGTGTAGGAACGAAAGGCTCAGGCTCAGAGAGTAGTCCTGCCGTCTCCTACCGACACCGATCCTGCAAGCTCGGGGTAAATGCGGACCAAGCCCACTGGAAGTAATATCTATGTAGGGCATAGGTCGGGCATAT GTCTCAGGGGGCGACAATCACTGCTTAGCCGATGCCTCAGTAACTGCGAGAATACTGCGGCTAGGGAGATCGAAAAAAAATTCGCTAACGGATGACTGAACTTCAACATGTACGCGGTCCTCTCTGTCACTGGACTTAGTCCATAGGTCGGTAGCG AGTAGGGCGTTGTTAGGAATCCTCGCCGGTGATAACGTTCGAACGGACATACGGGGACGCACAATAGGCTAAGGCCGGTATCTCTCTCTGTTATCGAGCTTTGGGCAATTCCTTGCGCCCGGGGTTGGTGTTCGACTAGTTTGGCGAAAGTGCTGG ACTTCCGGTTAAGGGCGTATAGGAGCTACGTGACGATCTCTGTTACAGCTACGGGACCGGATGAGTCAAGATACATGTTGGCCCGCTCACGGGTTATCCGCGGGGACGCAAATACCCTGTATCAGAAACGGGAGGTTCCGCACTTCGGCACTTACC GTGCAGCGTTCTGAGCTCGTGGCTTCGCCGAAAGTGCTAATACCACTCAAGGAGGCCGTGTCCACTAGGTGTCGCTTCCTTCTGAGGGGACCATTCCCGTCGGTTTGGAGGGGGTCTAGCGTCACTTCAGGCACTCTGTAAAAGAGAGCTAGTATA ATTTATACGGCTGGGCAGCCCCCAGTGATAATAGTGCTCAGGTACATATGTCTTGTTGCGATATACAACGGCGCAGTCCCCCTAGATTCTAAGCCCATACCATTGGGACCTCTGTGAGCGACGCCCAACAGACAGTACCGTGTTACATCTTTCAGT TTTTAATAGGAACCTTGTCCCGGCTTCTTAACTGATACCTGCATCTTACCATACCATCTTCAAAGACTCCCTCAATCCTGTTTACGGGCATGTTGGGCTCTATTACAGAAGAATTCTGCATCGGCCGCTGAGCCTCTGTAATAGAAACAGATTCGT".split()
k = 12
t = 25
best_motifs = improved_greedy_motif_search(Dna, k, t)
for motif in best_motifs:
    print(motif)

TCTCTGTTACTG
CCTCTGTGAGCG
ACTCTGTAAATG
ACTCTGTCAATG
CCTCTGTTATGG
ACTCTGTAAATG
GCTCTGTTAAAG
GCTCTGTCAGAG
GCTCTGTTAGGG
ACTCTGTAAATG
CCTCTGTGAGAG
CCTCTGTCAAGG
GCTCTGTGATTG
ACTCTGTGAGCG
TCTCTGTTAGGG
GCTCTGTGAGTG
ACTCTGTAATCG
TCTCTGTAAGGG
GCTCTGTAAGCG
TCTCTGTCACTG
TCTCTGTTATCG
TCTCTGTTACAG
ACTCTGTAAAAG
CCTCTGTGAGCG
CCTCTGTAATAG


**Randomized Motif Search**

In [1]:
import random

def Random_Motifs(Dna, k, t):
    motifs = []
    for i in range(t):
        start = random.randint(0, len(Dna[i]) - k)
        motifs.append(Dna[i][start:start+k])
    return motifs

In [2]:
def Generate_Motifs(Profile, Dna, k):
    motifs = []
    for seq in Dna:
        motifs.append(profile_most_probable_kmer(seq, k, Profile))
    return motifs

In [3]:
def Score(Motifs):
    consensus = ""
    k = len(Motifs[0])
    for j in range(k):
        count = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        for i in range(len(Motifs)):
            count[Motifs[i][j]] += 1
        consensus += max(count, key=count.get) 
        # Contains the sequence of the most frequent nucleotides at each position across all motifs
    
    score = 0
    for motif in Motifs:
        for i in range(k):
            if motif[i] != consensus[i]:
                score += 1
    return score

In [4]:
def Randomized_Motif_Search(Dna, k, t):
    Motifs = Random_Motifs(Dna, k, t)
    BestMotifs = Motifs
    while True:
        Profile = form_profile_pseudocounts(Motifs)
        Motifs = Generate_Motifs(Profile, Dna, k)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
        else:
            return BestMotifs

In [5]:
def Run_Randomized_Motif_Search(Dna, k, t, iterations):
    best_motifs = Randomized_Motif_Search(Dna, k, t)
    best_score = Score(best_motifs)
    
    for _ in range(iterations):
        motifs = Randomized_Motif_Search(Dna, k, t)
        current_score = Score(motifs)
        if current_score < best_score:
            best_motifs = motifs
            best_score = current_score
            
    return best_motifs

In [10]:
Dna = "GGAAGTGTAGTTTTCACATAATCTTGTGTGATTCTCTATAGGAGCATGATGCTTGTCTCGTACAAGGCTGGCAACTGCCATATGCCCACATACATCTTCCCTCTGCTTCGATACTAGGGTTGGCCGATATTGTTGTTGGCCTATAGATGAGTTTGCTAATGGAAGTGTAGTTTTC ACATAATCTTGTGTGATTCTCTATAGGAGCATGATGCTTGTCTCGTACAAGGCTGGCAACTGCCATATGCCCACATACATCTTCCCTCTGCTTCTGTGGCCGTCATCTTGATACTAGGGTTGGCCGATATTGTTGTTGGCCTATAGATGAGTTTGCTAATGGAAGTGTAGTTTTC ATAGCCCACTGTTAGGATCTACACCCCATCGTGTTTGCGAATTCTTGCCGGTTCGATATGGACTACGGATACAACGCAACACGCTCCGACGCACTAACTTACCCAGTCACTCACTTCAATGTCGTACTACCCACCTTGATGGTAGTCAGTGTCCAATCCTACCTCTCGCTTGTCT TATTACCCTCTGACTTGCTCGAGCTGCCCTCCCAGTGCGCTGCTAGAACCGCTACCCACAACGTCATACCATGTTTGTCCCATCTTGTACGGTGGGGAAGCATATTCTCGAGACTAGCCAAATCCGATCCTGCAGTATTACATCTTGGTTGCCCGATTGACTTGCCCCGTTACAA GGGTTAAGTAGGTATAACCCTCCACTCTGTGAACGTCATCTTAACCATCTGCAACATGATAATGGACTCAATGCCACTTCCAGGCTATCCAGTAGCAAAACTTGACAATACGAGAATACCACCGACTAATCCTAGGGTTGCTGAGTTTACGAGTGCAGCAGGAAACCCACTGCGT GTTAGATTACCCGTCCACGTTGAGACTGGCTCCAAACGCCTTGCCTATGTTTGCGTCATTAATTACGACGGTTTATTCTCGGTCAAGTTTGAGGTCTGACCCGGATTATCCACATAAAATGAGAGAGGCGTAATGTAAAACCTCAGGTGTTGTATCTCAACGGCGGTCGTAGTAG CCGCAGCATTAGCTAAACGTGGTTCCTAACACGGGTGTTCATCGTAGCAGCCCCGCAGGTCATACAAAGTCGCTTGGGATACTACCTGTTTTGTTCATCTTGCCGCAGTTAGAAACGGTGTCGATCTCCCTCTACTGTTAGAAGCGAAACAAACGCATGATTTGCCATACGCGGT TGATTTGAGTCCTATGCTTACATACGAACGCTCTCGTATTATATTATTTTGGAAGCGTCATCTTCAAAGTATTTGGTACCTGTGGATCTCTCTCTAACGTGCGACTGTAATATGGGGTTATCAGAAGGCAGGCGAGCCCCTATCTGCGGACGAGTCAAGGGGTGGAGGTGGGCGA GGCAATCAATTAACAGGCCACATGGCAAGCCTAAAGACTGCGCCAGCGACCTAGCGCGCCGTCCCCAGTTGAAAATCTTACCGCCCCCACTCGTAGCACAAAACTGTTTGCGTCAAGATGGGGGGACAGTCCAACCGGTAATGCCCGATGAGTTGTCGGAGGCTAAGCTCCCGGC CCGGGAACAGCTGGCAATACTTGGCAGTAGGACAACGACCCACAAAACTAAATCCGCTGTTCGTGTTTGCGTCGGGTTAAGTGGCCTATTATGCACTCTAGTATAGCCTATAGATTACTAGCGCTCTAGTGATCTACTCCGGATAGTGGGAATGATGAATCTATTCGTTGGTGAC AGTGAGCGAGTGACCTCTTATGTCTAGTAAGTGAACCGAGGAATAGTGCGCGTGTAAGAAGAATTCGGTGCAATAGTATTTCTACTTTCCGTCTCCTTTGCGTCATCTGTAGTAATGCTCAAAACTACTCTCATGTTGTAGCTTATCGCGACTTGGCTAATGGTCACCGCTGAAT GTACGCGGCGGATAGGCAATCCCCAATCGTGTATTGCGTCATCTTGCGACATCCAGTATGTCGCCCCAGCCGTGTTATCGTACTTCTTTAGCCACACTCTCAAACCGTGAGCATTCAACGCCATATTACGATATACTTGGGTGGGAGGTATCACCGCAGCGGGTCGCATAATTGG CAGTCAAGCTCTCATGCTGCGGGAGTGGGGTTTGTAGCGTTCCGGGTCATTAGTGGTGGGGTGCCTATGCTTCCTCTGGTTTATATTGTCAGTTAACACACATTGTTTGTCGCATCTTTCTTGTACTATACTAGGACGCAGCAGCGGCGCCACGCGCCCTGTCGAAGTAAAGGGG AATCGCCGATCCAAATGTTTGCCGTATCTTAAGAGCAGTGAGATGGTCTGCTCATGGGGCCTCTTAGCCATTCCGTGTGGACGGCTAGGCCGTAGCGACCATTATTATAACTCCATCCCGTCCGTGTAAACCCTAAATTCTTCACGCGTGCTGCTCATCGGTAGGTTACCGTATG TTTGTACTTTGTAGATCAGTTGTTTAGTAGGAGGCCTGTCTCCAGAAACGATTCGTTATCTGGTACATGAGCTATGACACGGATTAGAGAAATAAAATGGCACGTCCGGAACCACAGGCTGTTATTGTCATCTTCTATTCGCTGCTGCGAGGCTGTCTGCACGATGCTCAATGAG CTGGGGCGTCGTGGATCGAAAATATTGCTCAGGTTGGACGCCTGTATAATCATGTTGTGGTCATCTTGCCCCTTAGGCTTGAAGAGCCAGCTAATGGGCAGAACGAGGTCTTAAACATTAGAACCGCTATTCCGATCACTTGCGACTTAGGTACACGGGATTCATGGGTTATGTC TTGTTTTTATCATCTTAATCCAAGGACTAGCACCGTCGACGCCCGAGGAGTCCAACCGTGGCTCAATCACCCGATAACGGTCTCCCACTGAAGGGAATGTATTTCCTCCCGGTTACCCTATATTTGCTACCCGCGCCCACGGCCGTTGGGTTTTGCGAATTTAGGCGGGGGAAAA GTGGTAACACGTTTAATTTGCAACTACGTCTAGAGGCGCGGAAGAGCTGGACGTTGCCGCAATACGCTTTGATGCGACGCTACTTTCAGGTAAACCCGCAAGTCCTGTTCATTACGTCACTTCGATGGGTTATTCTGGAAATGCTTGGAATTCCTGCGTCATCTTGGATCACGAG GTGGTCCCTGTGAGTTATTCGGTTCAGCAGGGTGACGTTTGCGTCATCGATTTAGACCAAGGTCCGAGCGAAAGTCGACTTAATTGGCTAGTTCTTCAACGCAATTGGAATTTGCCGTTAAGGAAATTTGCGACAATAGGTAAGAGCCGAATTGCGGGGCCGACTTGTCTGGATG CAGCCTCACGTGAGGGGGAGCAGAGTAAAGCATCTTGAATTGCTACCTGAATAAGAGATAAGACGGAAATCTCAACGGTATACTTTGAACTAGTCTTAGTCACCAGATGTGGCGTCTACCTGTCAAGGCTTTGGGATGGGGTGTTTGCGTGGCCTTGATTAACTGAGACAAACAA".split()
output = Run_Randomized_Motif_Search(Dna, 15, 20, 1000)
for i in output:
    print(i)

TTTTCACATAATCTT
TGTGGCCGTCATCTT
TGTTTGCGAATTCTT
TGTTTGTCCCATCTT
TGTGAACGTCATCTT
TGTTTGCGTCATTAA
TGTTTTGTTCATCTT
TGGAAGCGTCATCTT
TGTTTGCGTCAAGAT
TGTTTGCGTCGGGTT
CCTTTGCGTCATCTG
GTATTGCGTCATCTT
TGTTTGTCGCATCTT
TGTTTGCCGTATCTT
TGTTATTGTCATCTT
TGTTGTGGTCATCTT
TGTTTTTATCATCTT
TTCCTGCGTCATCTT
CGTTTGCGTCATCGA
TGTTTGCGTGGCCTT


**Gibbs Sampling to find hidden Motifs**

In [31]:
def ProfileRandomlyGeneratedKmer(seq, profile, k):
    n = len(seq)
    probabilities = []
    for i in range(n - k + 1):
        kmer = seq[i:i+k]
        prob = 1
        for j in range(k):
            prob *= profile[kmer[j]][j]
        probabilities.append(prob)
    probabilities = [p / sum(probabilities) for p in probabilities]  # Normalize probabilities
    chosen_index = random.choices(range(n - k + 1), probabilities)[0]
    return seq[chosen_index:chosen_index + k]

In [51]:
def GibbsSampler(Dna, k, t, N):
    # Randomly select initial k-mers from each string in Dna
    Motifs = [seq[start:start + k] for seq in Dna for start in [random.randint(0, len(seq) - k)]]
    BestMotifs = Motifs.copy()
    
    for j in range(N):
        i = random.randint(0, t - 1)
        Profile = form_profile_pseudocounts(Motifs[:i] + Motifs[i+1:])
        Motifs[i] = ProfileRandomlyGeneratedKmer(Dna[i], Profile, k)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs.copy()
    
    return BestMotifs

In [52]:
def runGibbsSampler(Dna, k, t, N, runs = 100): # Increase the runs to converge properly
    bestMotifs = []
    bestScore = float('inf')
    
    for _ in range(runs):
        motifs = GibbsSampler(Dna, k, t, N)
        currentScore = Score(motifs)
        if currentScore < bestScore:
            bestScore = currentScore
            bestMotifs = motifs
    
    return bestMotifs

In [53]:
Dna = "CGTTCACGCTAGGACATCCGTTCCCCTGTAGTGCACGCACTGAACGAAGCTGAGGGGGTTCCGTCCCATTAAGTTCCCTCACCGGAATGCTATAAATCTAGGCTGCCCATTCGGATTAACGATGGCCATTCATCAACCGATCTGTACAATGCGTCGCCGACAGGTCTCCGGTCGTCCTACACGTCATGTCGACGATACCTAGAGGTTGCCGACCTAGATACACCATTGCGCGTATTCAGAGGGGATCAAGGTGATTGCGCCGTATCTCCAAGTGAAACATATATGTAAGGACACTTTTGCGCAATGAACCCGTTCACGCTAGGAC ATCCGTTCCCCTGTAGTGCACGCACTGAACGAAGCTGAGGGGGTTCCGTCCCATTAAGTTCCCTCACCGGAATGCTATAAATCTAGGCTGCCCATTCGGATTAACGATGGCCATTCATCAACCGATCTGTACAATGCGTCGCCGACAGGTCTCCGGTCGTCCTACACGTCATGTCGACGATACCTAGAACTGATGAGGGCAATGGTTGCCGACCTAGATACACCATTGCGCGTATTCAGAGGGGATCAAGGTGATTGCGCCGTATCTCCAAGTGAAACATATATGTAAGGACACTTTTGCGCAATGAACCCGTTCACGCTAGGAC TCAATAGCAGTTGCGGCAAATGCGCGACAACTGCCAGCACGAGCCACTGAAACTGGCAATAAGCTTGCTGACTATAGAGGGCTCGCGTTACATTGTGTACCACGGAGGGAAATTGCTCTACGCATAATGGAATCGATAGTAAATACATATCAGTTCGGCAGCTGCCGGGGATAGGCTTGGTCAGGACTACCAAATAATGATAATATCACCAGGCATAACCAGATATCTATCCACCAAGACCTCTACTCGGATTAGTCCTATCGAATTTCACGTGGTTATGGCTGCCGGACAGAGCGACAGGGAGACACCGGATTCATTCCAGTGT CAACCTCAGCGTTATGTCCCACGCTAAGCCCGTTGAGCGATGCACTTGAGTAGGCAACGAGAGATTGCGATATATTAGAGACAAACTCCAGAAGTCTGGATCGTGCATGCACGAAAAGGGCCCGATTGCTTTCACATGGTAACGAGTAGCGCACTCTACGAACGTTATAGAGGGCAACAATAGTACCTGCACACGCTGATACTTGCTGTGCCTCATAAGGCAAGCGGTCGATGAACCAATCGCCATAACATAAATCTAACAGATAGATTATGGCTGCCATATCTACGTTATGTAAGAGCTTCAAATCTTTACCAGGGCGCCATAT TGCTACGAAATGGTAACACTATAGAGAAGAATAAGTTGAATCTCGCCGCTAGCTTCATGGATTATTCTATTCGCAGTGACTGTCACCATCAGCACCAGTTTTGATATAACTATCTCACCTAACAAGGCTCCAAGTCAAGCTCGCTCGGGTCTCAGACCTTAACTAACGTAGGGCATGAGAGTTAACTGTGAACCTATGATGTGAGGCAATTCCGACAATAATGAGTAAGGGAACCTAACAGCGCAAGGCCACATTTAATCTGGTGTTACTACCTTCCCGGTTAGAATTTGCTGCAGCCTGCTAGATTGCCTTAAATCTGTCCAGA GACGTTCTTATCCTGATGATCAGCGCAAAGGCTTGCCGACGCGGACACTCTATCTAGGCTTGTTATTGCCGGGTACTACTCACAAGACGTTCGATCAATGTTTACTAGCTAGGGCAATGAATCTCTCCTCAGGCCATGAGGCCATACCAGATCAGTAACCCAGGCCACTCGTGCTCTGACTCATGGGATTTCTGGAAGCCGCGAGAAGATACCCAAAGTCCCATTCACCGGTTAATATTAGGGGCCGCGCTGCAGACTGAGAGCATCGCCTCAGAGGTTTACACCGCATCTAGCAATTTGGGTTTTGCTTCTGCTGATTACTCAC CATTCCAGGTCAAGTAATCGATATTTATGAGCCCCCCATCTCCTTATTCCCCTGCTCTAGCTTTGGTCTCTTTAGTGAACTGGGCGAGACTATAACCGGCAATTGCAGCTTGTCAAGACTACATGCACTGTACGAGAGGCGACAATGGGGAGAAGTCTAGGTTCAGTAACTACTATTCCGCGAAGGGCACTCCGCTATCCTTTACCGCACTCTTAACGTGATAACGGCCGACAGGGGAGAACCGATGCTCATGGACATCATCACCCATGTTCTATCATTCTTACGTTCATCCGCGCATGGATTCCCACGAATATGTTCAGATATC ATTACGCGAGAGGGCAATCGTGAGTCATTACTGCCGGCTGCTGTGGGTGAGGTGACGTTTCTCAACCCGTCGTATCTGTGTCCAACGTGCAAGCGGCCCTCGCCTATGGGGATTTACCTCTCCTTCTGATGATACGCCAGCATATTACGCTCTTCTCTCAAATCCTTAGATGATGGGTGGTTATTTAGAAGAGACGCAACATCGCCCGCTTCGATGACTATTTATCCCTGCAGATGGCTGACACAATGAGGTGCGGAGCGCAGGACGGCTACGCGGACGGAACCCAAATAGAGATGGGCGTACACAGGAGTATTCAAACGATGCC TGCGGGGGGGGTCGGCCGTTCTGTGAGTGTTGATCACCTAATTTGAACATGTTTGCAAATAATACTACCCATGTTGTCTAAAAGGGGCTATAATGATTCTTTTAACTTCGACGGCCATTCCTTCCGCACGTGTCTGTGCATGAGGATGCCGCTGCCCGGTCCTACCCCAAAGAACTGGGGGCCGATAAATTATAACACGTGGTCGGCTCCCAATTCTATCTAGTACGTGAGATGAATCGGTGGGACGGCTTAAAGACATAGAGGGCAATTTGAAGTGTTTCCCCCACACAGAGTTATGCGCACGGTAGATGCTTCCGGTTTACCG CCCAATATTGCGACATAGGCGCCGGAAGAACCCAGAATGACAGCACTGCTTCTCCACCGGTCTGTCTTTCGCGGGGTTCCGCCTGCTTAACATAATAAGGTAAAATAAGATGAGGTGACTGGTATAAAACCACTATTGTCTCTCCAAATCACGTCTTACAATCTTGACTATAGAGGGAGGTTAATGTCCACAATTAAAGTCAAAGACTGAACTAGGGAACTCGTCTACTTCATGGGAGACTGCTCCGCCCGTCGTAGTTCGGCTCAAATTCCTACGCTCGCCGGCCAAACATAATTTCACACTCGTAAACAAACGCGAGCTCTAG AGGATGTGAAACCAGTGTAGAGAGACTTGTTCGGACATCCACTATATCTGGCAATATCACTTTGGGTTCATCTCTCCGTGTTTACGATGGGGTCTGCACGCTTGACTAGCTCGGACGTCTGCACGTTGCAAAACATAGCTCGTAGTCTCGAACGATAGTAACTCTGTGTCCCAGTTGCGGACGAAATGATGTAGTACCGGACGCCTCATGAACCATTACAACAATTGATTGCACGCTAGCGATCCTTGACAATTAGGATAGTATTCCCATAAGGGGGCGCTAATAGTAATGCTCAGAACGGTCACCGCTTGGGACGGCCCTGATA TTATGAACGCATGTCGATTAACTCTTATGGCAACTATAGCCAGGCAAGGGTTATACTAGAGGGCAATGTACGCTAAACGTTTCTCGATGGAGTGCGTGTGCCTGACCCACCGAGATCCAGTTTCCCAGTCTTCGGAACACATTGGACCTGGCCGTAATAGCGTAGGTCAAAGGGGCTACCACCCTAGAATACCCGACCTAAAGGCTATTGTATGTTAGTGAATTCGCTCGAAACACTATCGTGAATAGTCCCTAGTAACATGAGATGTCTACTCCTAATTTATATTTTCATTTTGATTCATTATCTATACTCCGAGGGCTTACAG ACGAACCGTAGGGTGTCAACCAGTGCGACGGCGGTGCTGCCCTGTATTATGGAACTCAGGAGGGCAATCACGCTAAATACACAGGCAACGGGGGAAGCATCATCTCACAGAATCCTTTGTCAGTTGTTTTGCATTCAAGTAACCAGCCCCGCCGGTGCTAGCACGTGCTCGGGGGGTTATGGAGCGCAACAGGCTTAATACGCAACGAAGCTACTGAGTCACCGTACATCCCTGAAACGGTGTTGGAGCCCACTGTAAGTAATGGCTTAAAGTGGCCCCGGTTTGTGGAATTAATGCAAGGCCCAAGAGATATCGTACACCTTGG CAAGCCAACCATGGAGAGCGTTAATCTTTGGCCGTACCGTCGAAAAGGCCACGCGCGATCCAGTATAGTGCATAAATGTTAGGCGCGACGCGGTAAGTTTGACAAAACTGTCCAGCGGTGATCGTTGGCGAATCCCGCGGGTCGCATGAGATTGCGCGCCTGTGGCTCACTATAGAACTCAATCCAGAAAAGAGATTAGGAATACCTGATGGCCTAATACCTTCCTGTTCACGAGTAGTCTTTAATCGGCCAAAAGCTCTTCATGACTCCCTCTACGACGAGGCCTAACAATCAATGAGTGTGTAACCCGGTTTCACTCGAGGGG CGTTTGGAGCTCGATTGGCTAAGGCACAACTCTTGGGAGAGCTGGTGTGAGTACAAGAATCTGTTTATAATGCTAAGCTGACCTCTTATCGTATTAGCTTACTAGAAGTCGGCCTTACTTGCTTTCCAAAATGCCGGAGATGAGTGGAGCTGAGCGTAAGGTCACGTGATGTTACCCTATGGCCTTGTTGCATTATAGCTAGAGGAGAATGGATAAAAACAAACTACTATTTTGGGCAATCAGTGTGTGAGACAACAATTTGGGGTAACGGCGTCTGACTGTTGCTTAACAGTGTTGACCACGGTCACGACTGAAGGGGTTCAAG TAACAGCGGCGATAATTGGGGAGACTTGGGTTACACAACTAGAGATTCGACCCGTGTTGATTCCTCACGCCCGCGACAAAAAGATTTGACCGCCAATTTATCATACTCACAAAACGGTATTATGTTGTCGTGGAGCATAGACTGCTAATCAGTCAAAAGTGCCTTGATGTTCCAGCTGTCGACAAAGGGCCGGGTGGCTACGTGGCGCTATGTACTATAGTCCGCAATTCTTTTAATTTGCAGAACATGACTTCGCTAAGGAACTGAATACCGTCGCTATGCGGTGCGTTGATGGAATGCTATACTCTTATGTGTGCGCTACTCG AGATACAGGCAGGGGGCAATAGGCAAGAGATAGGTCCCGAGGAACTTTTTTGGCAGGTAGGTGCATCATGGGATTCCCCCTAGTGGAGTGCCACGTGGGACCCCGAGGCTTAAGGGAAGCGCGAAAGGCCACGTCGCGCAATATGCACCGTCAATTGGGCTTTTTAAATCGTGTTAGATCCTAACTGACACCTAAGTAAGCCCTGCTCCGCGCCGACCAACGCCGGACCGATGACTATTCTGGGCAATCTTATCAATGACTGGTACACCAGAACTTGATTCTGGAAGATGGGCGGAGGTTCGGGTGTTAGGGTTCACGACGGACG CTCGTTTTCATCTTAAGTACAGGCTCCAGGTTGGGAACTACTCAGGGCAATGATTTAGACTAAGGGATCCGTACACATCAACGATGTGTGGCAGTAGACCGCAGAGTTTATCAGACTGATCGTGAGAGATAGGGGGCATCAGGTTACGTCGGCGTAACTTGTAGAAGTATAGTGTTGAGTGTCGCGGCCTACCTAAATTATGGCGTCGGACCGTACAAACCGGCTCAACACGGCTGCGCAGCGTAGCTGGGCCCACCGTCGTTGGCGCCTGTCCTTCGCTTGGACCCAGGCAACCTCTGTGTCCTCCGATGGCCAATGCTGACCA GGTAAAGGGTGGAGTTCTTGCAGTGGACAAAAATCTCCACCCATGGTGTACCGACTACGCATAGGGAGTCGTGCGTTGCCTACGGTGTTTAGCGTCGTTATCGGTCGCCGAGTAGCTTAAGATTCCTAACGCTTGAGATAGGATACGCTCTCGCAAGATCTCCGGACCCATTAAAGACTACTTTTGAATGTCTGCGCGGATCTGCCAAAAGCAAGTACGTAGCACGCGATTATCGCATCTGACCCTACAACATATAAATCAGCCTTTTTGGAGGTCCGAAACAGGCTATAGAGGGCACACCCCGGCGCTTCGCAGTCTTGTTCAA CACATGCCCCGCAATTTGGCTATACGGGCCGCCGTAAGAGGCGCAGGTGCCTGCATCCCCTCAAGCGTCTAGGGCACGTTACGTTTTTGAAGTGGAAGACCGAGCGAAGGATTATTTTCGAGGCTCGCCTCCTGAGTAGGATCCCCCCCAACTGCTTGAGTTACTTGTGCCCTTATCAACTATAGAGGTACATATTCGGGCTTTTGATCGCTATAACGCATCAAACGCTCCAACAAATTTATACGAGTCGTCTCTTTTGTTGGGATTTCAATAAGTCCAACGGGAGCTGCAAGCGACCATGACTCTGCCACGTGACCGCAAACAT".split()
output = runGibbsSampler(Dna, 15, 20, 2000)
for i in output:
    print(i)

ATTAACGATGGCCAT
ACTGATGAGGGCAAT
ACTATAGAGGGCTCG
GTTATAGAGGGCAAC
ACTATAGAGAAGAAT
ACTAGCTAGGGCAAT
ACTATAACCGGCAAT
ACGCGAGAGGGCAAT
GACATAGAGGGCAAT
ACTATAGAGGGAGGT
ACTATATCTGGCAAT
ATACTAGAGGGCAAT
ACTCAGGAGGGCAAT
ACTATAGAACTCAAT
ACTATTTTGGGCAAT
ACTATAGTCCGCAAT
ACTATTCTGGGCAAT
ACTACTCAGGGCAAT
GCTATAGAGGGCACA
ACTATAGAGGTACAT
